## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-15-08-57-38 +0000,nyt,Australian Police Investigating Sydney Shootin...,https://www.nytimes.com/live/2025/12/14/world/...
1,2025-12-15-08-52-00 +0000,wsj,Decades of Tight Gun Laws Failed to Prevent Sy...,https://www.wsj.com/world/europe/decades-of-ti...
2,2025-12-15-08-51-00 +0000,wsj,Gold Rises as Traders Await U.S. Data,https://www.wsj.com/economy/central-banking/go...
3,2025-12-15-08-50-17 +0000,nyt,How the Bondi Beach Community Rushed to Help S...,https://www.nytimes.com/2025/12/15/world/asia/...
4,2025-12-15-08-49-00 +0000,wsj,Sterling Could Rise Strongly if Data Are Upbeat,https://www.wsj.com/finance/currencies/asian-c...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2355/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
19,bondi,31
4,shooting,27
20,beach,27
56,brown,20
246,trump,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
166,2025-12-14-18-41-53 +0000,nypost,President Trump mourns victims of Bondi Beach ...,https://nypost.com/2025/12/14/us-news/presiden...,156
27,2025-12-15-06-37-34 +0000,wapo,Live updates: 15 killed in Bondi Beach shootin...,https://www.washingtonpost.com/world/2025/12/1...,122
179,2025-12-14-16-36-41 +0000,nypost,"Famed Australian rabbi Eli Schlanger, who just...",https://nypost.com/2025/12/14/world-news/famed...,121
88,2025-12-15-00-36-03 +0000,latimes,Los Angeles Jewish groups ramp up Hanukkah fes...,https://www.latimes.com/california/story/2025-...,111
184,2025-12-14-15-32-12 +0000,nypost,Bondi Beach terror survivor describes how cops...,https://nypost.com/2025/12/14/world-news/bondi...,110


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
166,156,2025-12-14-18-41-53 +0000,nypost,President Trump mourns victims of Bondi Beach ...,https://nypost.com/2025/12/14/us-news/presiden...
71,69,2025-12-15-02-16-02 +0000,nypost,"Legendary director Rob Reiner, wife found stab...",https://nypost.com/2025/12/14/us-news/2-dead-a...
13,54,2025-12-15-08-03-55 +0000,bbc,Pro-democracy Hong Kong tycoon Jimmy Lai convi...,https://www.bbc.com/news/articles/cp844kjj37vo...
179,54,2025-12-14-16-36-41 +0000,nypost,"Famed Australian rabbi Eli Schlanger, who just...",https://nypost.com/2025/12/14/world-news/famed...
189,39,2025-12-14-13-40-25 +0000,cbc,"Australian police suspect father, son behind d...",https://www.cbc.ca/news/world/bondi-beach-shoo...
178,33,2025-12-14-17-00-00 +0000,wsj,"For decades, Tonka trucks have been a family s...",https://www.wsj.com/economy/trade/tonka-trucks...
76,31,2025-12-15-01-51-45 +0000,nypost,Mom says viral video shows her being booted fr...,https://nypost.com/2025/12/14/us-news/mom-says...
115,28,2025-12-14-22-16-27 +0000,nypost,Australia’s PM blamed for failing to stand up ...,https://nypost.com/2025/12/14/world-news/austr...
174,28,2025-12-14-17-12-07 +0000,nyt,Chile Poised for Right-Wing Victory as Crime F...,https://www.nytimes.com/2025/12/13/world/ameri...
98,26,2025-12-14-23-43-16 +0000,nypost,Holocaust survivors and children among the 15 ...,https://nypost.com/2025/12/14/world-news/holoc...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
